In [1]:
import pathlib
import shutil
from datetime import datetime, timedelta
import os # 파일/디렉토리 삭제 및 생성용

class FileOrganizer:
    def __init__(self, target_directory: str):
        self.target_dir = pathlib.Path(target_directory)
        # 파일 확장자와 카테고리 매핑 정의
        self.extensions_map = {
            'images': ['.jpg', '.jpeg', '.png', '.gif', '.bmp', '.tiff', '.webp'],
            'documents': ['.pdf', '.doc', '.docx', '.txt', '.rtf', '.xls', '.xlsx', '.ppt', '.pptx', '.csv', '.md'],
            'videos': ['.mp4', '.avi', '.mkv', '.mov', '.wmv', '.flv', '.webm'],
            'audio': ['.mp3', '.wav', '.aac', '.flac', '.ogg'],
            'archives': ['.zip', '.rar', '.7z', '.tar', '.gz', '.bz2'],
            'code': ['.py', '.js', '.html', '.css', '.java', '.cpp', '.c', '.cs', '.php'],
            'executables': ['.exe', '.msi', '.dmg', '.app']
        }
        # 모든 확장자를 쉽게 찾기 위해 역으로 매핑 생성
        self.extension_to_type = {ext: ftype for ftype, ext_list in self.extensions_map.items() for ext in ext_list}

    def _get_file_category(self, file_extension: str) -> str | None:
        """파일 확장자를 받아 카테고리 이름을 반환합니다. 없으면 None."""
        return self.extension_to_type.get(file_extension)

    def organize_by_category(self):
        """파일 확장자에 따라 카테고리별 디렉토리로 파일을 이동시킵니다."""
        if not self.target_dir.is_dir():
            print(f"오류: 대상 디렉토리를 찾을 수 없거나 디렉토리가 아닙니다: {self.target_dir}")
            return

        print(f"'{self.target_dir}' 디렉토리의 파일들을 카테고리별로 정리합니다...")
        moved_files = 0
        for item_path in self.target_dir.iterdir():
            # 파일인 경우에만 처리
            if item_path.is_file():
                extension = item_path.suffix.lower() # 확장자를 소문자로 가져오기
                category = self._get_file_category(extension)
                
                if category:
                    # 카테고리별 대상 디렉토리 경로 생성
                    category_dir = self.target_dir / category
                    # 디렉토리 생성 (이미 있으면 무시)
                    category_dir.mkdir(exist_ok=True)
                    
                    # 파일을 카테고리 디렉토리로 이동
                    destination = category_dir / item_path.name
                    try:
                        # shutil.move는 파일 이동에 사용 (os.rename과 유사하지만 다른 드라이브도 가능)
                        shutil.move(str(item_path), str(destination))
                        print(f"  이동: '{item_path.name}' -> '{category}/'")
                        moved_files += 1
                    except Exception as e:
                        print(f"  파일 이동 오류 '{item_path.name}': {e}")
        
        print(f"\n총 {moved_files}개의 파일을 카테고리별로 정리했습니다.")

    def organize_by_date_folder(self):
        """파일의 수정 날짜(YYYY-MM)별로 폴더를 만들고 파일을 이동시킵니다."""
        if not self.target_dir.is_dir():
            print(f"오류: 대상 디렉토리를 찾을 수 없거나 디렉토리가 아닙니다: {self.target_dir}")
            return

        print(f"'{self.target_dir}' 디렉토리의 파일들을 날짜별 폴더로 정리합니다...")
        moved_files = 0
        for item_path in self.target_dir.iterdir():
            if item_path.is_file():
                try:
                    # 파일의 수정 시간(mtime)을 이용
                    mtime_timestamp = item_path.stat().st_mtime
                    mod_datetime = datetime.fromtimestamp(mtime_timestamp)
                    # 날짜 폴더 이름 형식 (YYYY-MM)
                    date_folder_name = mod_datetime.strftime('%Y-%m')
                    
                    # 날짜별 대상 디렉토리 경로 생성
                    date_dir = self.target_dir / date_folder_name
                    date_dir.mkdir(exist_ok=True)
                    
                    # 파일을 날짜 디렉토리로 이동
                    destination = date_dir / item_path.name
                    shutil.move(str(item_path), str(destination))
                    print(f"  이동: '{item_path.name}' -> '{date_folder_name}/'")
                    moved_files += 1
                except Exception as e:
                    print(f"  파일 '{item_path.name}' 날짜별 정리 오류: {e}")

        print(f"\n총 {moved_files}개의 파일을 날짜별 폴더로 정리했습니다.")

    def clean_old_files(self, days_threshold: int):
        """지정된 일수(days_threshold)보다 오래된 파일을 삭제합니다 (수정 날짜 기준)."""
        if not self.target_dir.is_dir():
            print(f"오류: 대상 디렉토리를 찾을 수 없거나 디렉토리가 아닙니다: {self.target_dir}")
            return
            
        cutoff_date = datetime.now() - timedelta(days=days_threshold)
        deleted_count = 0
        
        print(f"'{self.target_dir}' 디렉토리에서 {days_threshold}일보다 오래된 파일을 삭제합니다...")
        
        # target_dir 아래의 모든 파일과 디렉토리를 재귀적으로 검색 (rglob('*'))
        for item_path in self.target_dir.rglob('*'):
            # 파일인 경우에만 처리
            if item_path.is_file():
                try:
                    file_mtime_timestamp = item_path.stat().st_mtime
                    file_mod_datetime = datetime.fromtimestamp(file_mtime_timestamp)
                    
                    # 파일 수정 날짜가 기준 날짜보다 이전이면 삭제
                    if file_mod_datetime < cutoff_date:
                        item_path.unlink() # 파일 삭제
                        print(f"  삭제: '{item_path}' (수정일: {file_mod_datetime.strftime('%Y-%m-%d')})")
                        deleted_count += 1
                except Exception as e:
                    print(f"  파일 삭제 오류 '{item_path}': {e}")

        print(f"\n총 {deleted_count}개의 오래된 파일이 삭제되었습니다.")

# --- 사용 예시 ---
if __name__ == "__main__":
    # 테스트용 임시 디렉토리 및 파일 생성
    test_cleanup_paths = []
    organizer_dir_name = "files_to_organize"
    organizer_dir = pathlib.Path(organizer_dir_name)
    organizer_dir.mkdir(exist_ok=True)
    test_cleanup_paths.append(organizer_dir)

    # 카테고리별 테스트 파일
    categories = ['images', 'documents', 'archives']
    for cat in categories:
        (organizer_dir / cat).mkdir(exist_ok=True)
    (organizer_dir / 'logo.png').touch() # 이미지
    (organizer_dir / 'report.docx').touch() # 문서
    (organizer_dir / 'backup.zip').touch() # 압축파일
    (organizer_dir / 'script.py').touch() # 코드 (카테고리 없음)
    (organizer_dir / 'notes.txt').touch() # 문서
    
    # 날짜별 테스트 파일 (수정 시간 조절)
    # 현재 시간 기준 2달 전 파일
    two_months_ago = datetime.now() - timedelta(days=65)
    ts_two_months_ago = two_months_ago.timestamp()
    (organizer_dir / 'old_file.log').touch()
    os.utime(organizer_dir / 'old_file.log', (ts_two_months_ago, ts_two_months_ago)) # Access, Mod time 설정

    # 현재 시간 기준 10일 전 파일
    ten_days_ago = datetime.now() - timedelta(days=10)
    ts_ten_days_ago = ten_days_ago.timestamp()
    (organizer_dir / 'recent_log.log').touch()
    os.utime(organizer_dir / 'recent_log.log', (ts_ten_days_ago, ts_ten_days_ago))
    
    print(f"테스트용 파일들이 '{organizer_dir}'에 생성되었습니다.")

    # FileOrganizer 인스턴스 생성
    organizer = FileOrganizer(organizer_dir_name)

    # 1. 카테고리별 정리 실행
    organizer.organize_by_category()
    
    # 2. 날짜별 정리 실행 (주의: 위에서 카테고리로 이동된 파일들은 현재 디렉토리에 없으므로,
    #    날짜별 정리를 하려면 카테고리별 정리를 실행하기 전 상태여야 합니다.
    #    여기서는 예시를 위해 각각 독립적으로 실행합니다.)
    
    # 날짜별 정리를 위해 파일들을 다시 원상 복구 (테스트 목적)
    print("\n날짜별 정리를 위해 파일 재배치 (테스트 목적)...")
    for item in organizer_dir.iterdir():
        if item.is_file():
            shutil.move(str(item), str(organizer_dir / item.name)) # 원래 위치로
    
    organizer.organize_by_date_folder()

    # 3. 오래된 파일 삭제 (예: 30일보다 오래된 파일 삭제)
    organizer.clean_old_files(days_threshold=30)
    
    # 테스트 완료 후 생성된 디렉토리 및 파일 정리
    print("\n테스트 완료. 정리 중...")
    try:
        shutil.rmtree(organizer_dir)
        print(f"테스트 디렉토리 '{organizer_dir}' 삭제 완료.")
    except Exception as e:
        print(f"테스트 디렉토리 삭제 오류: {e}")



테스트용 파일들이 'files_to_organize'에 생성되었습니다.
'files_to_organize' 디렉토리의 파일들을 카테고리별로 정리합니다...
  이동: 'report.docx' -> 'documents/'
  이동: 'script.py' -> 'code/'
  이동: 'backup.zip' -> 'archives/'
  이동: 'logo.png' -> 'images/'
  이동: 'notes.txt' -> 'documents/'

총 5개의 파일을 카테고리별로 정리했습니다.

날짜별 정리를 위해 파일 재배치 (테스트 목적)...
'files_to_organize' 디렉토리의 파일들을 날짜별 폴더로 정리합니다...
  이동: 'old_file.log' -> '2025-08/'
  이동: 'recent_log.log' -> '2025-09/'

총 2개의 파일을 날짜별 폴더로 정리했습니다.
'files_to_organize' 디렉토리에서 30일보다 오래된 파일을 삭제합니다...
  삭제: 'files_to_organize/2025-08/old_file.log' (수정일: 2025-08-04)

총 1개의 오래된 파일이 삭제되었습니다.

테스트 완료. 정리 중...
테스트 디렉토리 'files_to_organize' 삭제 완료.
